In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
from numpy import newaxis
import scipy.sparse as sps
from scipy.sparse.linalg import spsolve
import matplotlib.pyplot as plt

In [ ]:
from pyfem.topo import Interval
from pyfem.poly import gll_points
from pyfem.sem import SEMhat
from pyfem.poly import eval_lagrange_d0 as eval_phi1d

In [ ]:
from poly import eval_P

In [ ]:
order = 3
semh = SEMhat(order)

N = 30
n_dofs = (order+1)*N

In [ ]:
L = 1.0

vertices  = np.linspace(0, L, N+1)
EtoV      = np.zeros((N, 2), dtype=np.int)
EtoV[:,0] = np.arange(N)
EtoV[:,1] = np.arange(N)+1

topo  = Interval()
xq = topo.ref_to_phys(vertices[EtoV], semh.xgll)
jacb_det = topo.calc_jacb(vertices[EtoV])[0]

EtoV[-1,-1] = EtoV[0,0]
jacb_det

In [ ]:
# Make elem to dof map
EtoD = np.arange(N*(order+1))
EtoD = EtoD.reshape((N, -1))

dof_phys = xq.ravel()

# Averaging operator
rows = EtoD[:,[0,-1]].ravel()
cols = EtoV.ravel()
vals = np.ones_like(cols)

FtoD = sps.coo_matrix((vals, (rows, cols)),
                       shape=(n_dofs,N)).tocsr()
AVG = FtoD.dot(FtoD.T)/2.0

# Extract face DOFS
vals = np.ones(len(rows))
FD = sps.coo_matrix((vals, (rows, rows))).tocsr()
# Set face signs
vals[::2] = -1
SD = sps.coo_matrix((vals, (rows, rows))).tocsr()

# Jump operator
JUMP = FtoD.dot(SD.dot(FtoD).T)

In [ ]:
# Build Advection operator
S = sps.kron(sps.eye(N), semh.Ch).tocsr()

# Differentiation matrix
Dr = sps.kron(sps.eye(N), semh.Dh)/jacb_det
Dr = Dr.tocsr()

# Build full elemental mass matrix
x, w = topo.get_quadrature(order+1)
P = eval_phi1d(semh.xgll, x).T
G = sps.dia_matrix((w, 0), shape=(len(x), len(x)))
Bf = P.T.dot(G.dot(P))*jacb_det
Bfinv = np.linalg.inv(Bf)
B = sps.kron(sps.eye(N), Bf)

# Using trick from book
V = eval_P(order, semh.xgll).T
Minv = V.dot(V.T)/jacb_det
Binv = sps.kron(sps.eye(N), Minv).tocsr()

In [ ]:
tau = 1.0

In [ ]:
FLUXU = AVG
Q = Dr-Binv.dot(SD.dot(FD-FLUXU))
FLUXQ = AVG.dot(Q)-tau*JUMP 
A = S.dot(Q)-SD.dot(FD.dot(Q)-FLUXQ)

In [ ]:
def func(x):
    return -np.sin(2*np.pi*x)*(2*np.pi)**2

def exact(x):
    return np.sin(2*np.pi*x)    

In [ ]:
rhs = B.dot(func(dof_phys))
sol = sps.linalg.spsolve(A, rhs)
sol -= np.mean(sol)
plt.plot(dof_phys, sol)
plt.plot(dof_phys, exact(dof_phys))

In [ ]:
err = sol-exact(dof_phys)
L2_err = np.sqrt(err.dot(B.dot(err)))
L2_err, np.max(np.abs(err))

In [ ]:
# #errs = {}

# errs[N] = L2_err

# ka = sorted(errs.keys())
# va = [errs[i] for i in ka]
# plt.loglog(ka, va)

# if len(va)>1:
#     print np.log2(va[-1]/va[-2])